In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split # деление на обучающую и проверочную выборку
from sklearn.preprocessing import OneHotEncoder # кодирует целочисленные значения у в массив
from sklearn.metrics import accuracy_score # подсчет точности предсказаний
from sklearn.preprocessing import StandardScaler

# Подготовка данных

In [2]:
df = pd.read_csv('data/train.csv')

df_X = df.drop('label', axis=1) 
df_y = df['label']

train_x, test_x, train_y, test_y = train_test_split(df_X, df_y, test_size=0.1, random_state=20) 

# отложенные данные для теста
X_test = np.array(test_x) / 255 #  масштабируем чтобы значения были в даипозоне [0 - 1]
y_test = np.array([test_y]).T


# данные для обучения модели 
X = np.array(train_x) / 255 #  масштабируем чтобы значения были в даипозоне [0 - 1]
y = np.array([train_y]).T

y_onehotenc = OneHotEncoder().fit_transform(y).toarray()

'''
После всех преобразований получили размерности:
X - [29400x784]
y - [29400x1]
y_onehotenc - [29400x10]

x_test - [12600x784]
y_test - [12600x1]
'''


'\nПосле всех преобразований получили размерности:\nX - [29400x784]\ny - [29400x1]\ny_onehotenc - [29400x10]\n\nx_test - [12600x784]\ny_test - [12600x1]\n'

# Функция активации и ее производная

In [3]:
# ф-я активации, при флаге True возвращает производную (для подсчета градиента)
def sigmoid(x, derivative=False):
    if derivative == True:
        return x * (1 - x)
    return 1 / (1 + np.exp(-x))

# Функция ошибки и ее производная

In [4]:
# ф-я ошибки, при флаге True возвращает производную (для подсчета градиента)
def loss(y, y_, derivative=False):
    if derivative == True:
        return y - y_
    return (y - y_)**2

# Математическое обоснование

Описание параметров:
 - Ф-я активации (activation) - $SIGMOID$ (logistic regression)
 - Ф-я ошибки (loss) - $MSE$ (mean squared error)
 - Оптимизатор (optimizer) - $SGDm$ (Stochastic gradient descent)
 - Размер батча обучения (batch) - 32
 - Используется инерционность ($momentum$)


Ф-я активации СИГМОИД ($A$): $ \frac{1}{(1+e^{-\alpha})}$, где $\alpha$ считается как $ X_{input} * w + b$

Производная ф-ии активации: $dA = \alpha*(1-\alpha)$

Ф-я ошибки MSE ($E$): $\frac{\sum{(y-y^{'})^2}}{n}$, где $y$ - правильный ответ, а $y^{'}$ - предсказания сети

Производная ф-ии ошибки MSE: $dE = \frac{-2 (y-y^{'})}{n}$

Рекуретное отношение последнего слоя: $ \delta_{n} = dE_n * dA_n$

Рекуретное отношение скрытого слоя: $ \delta_{n} = \delta_{n+1} * w_{n+1} * dA$

Частная производная ф-ии ошибки по весам $w$:  $\frac{dE}{dw} = \delta * X_{input}$ --- градиент весов


Частная производная ф-ии ошибки по биасам $b$:  $\frac{dE}{db} = \delta$  --- градиент биасов

Уравнение обновления весов и биасов ($SGDm$): $\omega = \omega - \alpha * \triangledown \omega$ => $\omega = \omega + \triangledown \omega$, т.к. антиградиент с "-" и $-\alpha$  дают "+". $\alpha$ заменяем learning rate

Добавить в формулу обновления весов - $w = w_{old} + lr * (w_{delta} + momentum * w_{old}) $

# Архитектура и обучение NN (3 слоя)

In [60]:
epoches = 151 # кол-во эпох обучения
batch = 32 # размер батча
lr = 1e-1 # learning rate
momentum = 0.9 # коэффициент инерционности 

dE_dw1_old = 0
dE_db1_old = 0
dE_dw2_old = 0
dE_db2_old = 0
dE_dw3_old = 0
dE_db3_old = 0


w1 = np.random.random((784, 128)) / 1e4   # рандомно создаем матрицу весов для 1-го слоя и масштабируем
w2 = np.random.random((128, 32)) / 1e4   # рандомно создаем матрицу весов для 2-го слоя и масштабируем
w3 = np.random.random((32, 10)) / 1e4   # рандомно создаем матрицу весов для 3-го слоя и масштабируем

b1 = np.zeros((1,128))  # создаем матрицу нулевых биасов для 1-го слоя
b2 = np.zeros((1,32))  # создаем матрицу нулевых биасов для 2-го слоя
b3 = np.zeros((1,10))  # создаем матрицу нулевых биасов для 3-го слоя


for _ in range(epoches):
    for i in range(X.shape[0] // batch):
        batch_ids = np.random.choice(X.shape[0], batch, replace=False)
        
        y1 = sigmoid(X[batch_ids] @ w1 + b1) # предсказание 1-го слоя
        y2 = sigmoid(y1 @ w2 + b2) # предсказание 2-го слоя
        y3 = sigmoid(y2 @ w3 + b3) # предсказание 3-го слоя
        
        dE = loss(y_onehotenc[batch_ids], y3, True)  # берем производную по ф-ии ошибки

        # анти-градиент последнего слоя
        tmp3 = dE * sigmoid(y3, True) # рекурентное соотношение
        dE_dw3 = y2.T @ tmp3 # градиент последнего слоя с нормированием по размеру батча
        dE_db3 = np.mean(tmp3, axis=0) # градиент биасов последнего слоя

        # анти-градиент 2 слоя
        tmp2 = (tmp3 @ w3.T) * sigmoid(y2, True) # рекурентное соотношение
        dE_dw2 = y1.T @ tmp2 # градиент скрытого слоя нормированный по размеру батча
        dE_db2 = np.mean(tmp2, axis=0) # градиент биасов скрытого слоя
        
        # анти-градиент 1 слоя
        tmp1 = (tmp2 @ w2.T) * sigmoid(y1, True) # рекурентное соотношение
        dE_dw1 = X[batch_ids].T @ tmp1 # градиент скрытого слоя нормированный по размеру батча
        dE_db1 = np.mean(tmp1, axis=0) # градиент биасов скрытого слоя

        # обновляем веса и биасы
        w1 += lr * (dE_dw1 + momentum * dE_dw1_old) 
        b1 += lr * (dE_db1 + momentum * dE_db1_old)
        w2 += lr * (dE_dw2 + momentum * dE_dw2_old) 
        b2 += lr * (dE_db2 + momentum * dE_db2_old) 
        w3 += lr * (dE_dw3 + momentum * dE_dw3_old) 
        b3 += lr * (dE_db3 + momentum * dE_db3_old)
        
        dE_dw1_old = dE_dw1
        dE_db1_old = dE_db1
        dE_dw2_old = dE_dw2
        dE_db2_old = dE_db2
        dE_dw3_old = dE_dw3
        dE_db3_old = dE_db3

    
    if (_ % 10) == 0:  # показываем MSE каждые 10 эпох обучения
        error = y_onehotenc - sigmoid(sigmoid(sigmoid(X @ w1 + b1) @ w2 + b2) @ w3 + b3)
        print(f"Epoch {_}. Error:" +  str(np.mean(np.abs(error))))
    
        # считаем точность на обученной выборке
        train_predict_matrix = sigmoid(sigmoid(sigmoid(X @ w1 + b1) @ w2 + b2) @ w3 + b3)
        train_predict_vector = train_predict_matrix.argmax(axis=1).T
        print("Train score: ", accuracy_score(y, train_predict_vector))
        
        # проверка точности на отложенной выборке
        test_predict_matrix = sigmoid(sigmoid(sigmoid(X_test @ w1 + b1) @ w2 + b2) @ w3 + b3)
        test_predict_vector = test_predict_matrix.argmax(axis=1).T
        print("Test score: ",  accuracy_score(y_test, test_predict_vector))   # without bias 0.901984126984127
        
        # разделитель
        print("___________________________________________")

Epoch 0. Error:0.1800619564622661
Train score:  0.09656084656084656
Test score:  0.09833333333333333
___________________________________________
Epoch 10. Error:0.009487894921538566
Train score:  0.9777513227513227
Test score:  0.9635714285714285
___________________________________________
Epoch 20. Error:0.003417135077640958
Train score:  0.9935978835978836
Test score:  0.9702380952380952
___________________________________________
Epoch 30. Error:0.0017092196479927256
Train score:  0.9962433862433863
Test score:  0.9716666666666667
___________________________________________
Epoch 40. Error:0.00126747911939265
Train score:  0.9968253968253968
Test score:  0.9730952380952381
___________________________________________
Epoch 50. Error:0.0010170488646845418
Train score:  0.997010582010582
Test score:  0.9723809523809523
___________________________________________
Epoch 60. Error:0.0009043226751913659
Train score:  0.9971164021164021
Test score:  0.9733333333333334
______________________

In [53]:
df_test = pd.read_csv('data/test.csv')

test = np.array(df_test) / 255

    
predict_matrix = sigmoid(sigmoid(sigmoid(test @ w1 + b1) @ w2 + b2) @ w3 + b3)
predict_vector = predict_matrix.argmax(axis=1).T

# pd.DataFrame(predict_vector, columns=['ImageId', 'Label']).to_csv('submission.csv')

predicted_df = pd.DataFrame(predict_vector,
                            index = np.arange(1, predict_vector.shape[0] + 1),
                            columns=['Label'])
predicted_df.to_csv('submission.csv', index_label='ImageId')

C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
